In [1]:
from BayesianHoldem import BayesianHoldem
import torch

model = BayesianHoldem()

for i in range(10):
    card_representation = torch.randn(6, 13, 4)
    action_representation = torch.randn(24, 4, 4)
    prediction = model.predict_action(action_representation, card_representation)
    if prediction == 0:
        print(f"prediction {i}: Fold")
    elif prediction == 1:
        print(f"prediction {i}: Check/Call")
    elif prediction == 2:
        print(f"prediction {i}: Bet/Raise (BB)")
    elif prediction == 3:
        print(f"prediction {i}: All-in")

prediction 0: Fold
prediction 1: Check/Call
prediction 2: Bet/Raise (BB)
prediction 3: Bet/Raise (BB)
prediction 4: All-in
prediction 5: All-in
prediction 6: Fold
prediction 7: All-in
prediction 8: All-in
prediction 9: Fold


In [6]:
import pokerkit as pk

big_blind = 100
small_blind = 0.5*big_blind
blinds = [small_blind, big_blind]
ante = 0
min_bet = 0.5*big_blind
player_count = 2

P1_stack = 100*big_blind
P2_stack = 50*big_blind

game = pk.NoLimitTexasHoldem(
    # Automations - everything is automatic except for player actions
    (
        pk.Automation.ANTE_POSTING,
        pk.Automation.BET_COLLECTION,
        pk.Automation.BLIND_OR_STRADDLE_POSTING,
        pk.Automation.CARD_BURNING,
        pk.Automation.HOLE_DEALING,
        pk.Automation.BOARD_DEALING,
        pk.Automation.HOLE_CARDS_SHOWING_OR_MUCKING,
        pk.Automation.HAND_KILLING,
        pk.Automation.CHIPS_PUSHING,
        pk.Automation.CHIPS_PULLING,
    ),
    ante_trimming_status=True, # irrelevant but necessary for the game to be created
    raw_antes=ante,
    raw_blinds_or_straddles=blinds,
    min_bet=min_bet,
)

state = game(raw_starting_stacks=[P1_stack, P2_stack], player_count=player_count)
print(type(state))
print(f"Street index: {state.street_index}")
print(f"Stacks: {state.stacks}")

<class 'pokerkit.state.State'>
Street index: 0
Stacks: [9900, 4950.0]


In [7]:
print(type(state.operations[-1]))
state.complete_bet_or_raise_to(500)
state.check_or_call()
print(type(state.operations[-1]))
state.check_or_call()
state.check_or_call()
print(type(state.operations[-1]))


<class 'pokerkit.state.HoleDealing'>
<class 'pokerkit.state.BoardDealing'>
<class 'pokerkit.state.BoardDealing'>


In [3]:
print(f"Bets: {state.bets}")
print(f"Stacks: {state.stacks}")
print(f"Actor indices: {state.actor_indices}")


state.complete_bet_or_raise_to(state.stacks[state.actor_index] + state.bets[state.actor_index])
print(f"All in")
print(f"Bets: {state.bets}")
print(f"Stacks: {state.stacks}")
print(f"Actor indices: {state.actor_indices}")

state.check_or_call()
print("Check or call")
print(f"Bets: {state.bets}")
print(f"Stacks: {state.stacks}")
print(f"Actor indices: {state.actor_indices}")

print(f"Street index: {state.street_index}")

Bets: [100, 50.0]
Stacks: [9900, 4950.0]
Actor indices: deque([1, 0])
All in
Bets: [100, 5000.0]
Stacks: [9900, 0.0]
Actor indices: deque([0])
Check or call
Bets: [0, 0]
Stacks: [5000.0, 10000.0]
Actor indices: deque([])
Street index: None


In [1]:
from SelfPlayPokerGame import SelfPlayPokerGame

game = SelfPlayPokerGame()

for i in range(100):
    print(f"===============")
    print(f" New Game {i+1}")
    print("===============")
    game.run_game(verbose=True)

 New Game 1
======= New Round =======
Player 0 cards: [Kh, 5s], chips: 9900
Player 1 cards: [3h, Js], chips: 9950.0
Street index: 0
Public cards: []
Bets: P0: 100, P1: 50.0, Chips: P0: 9900, P1: 9950.0
Player 1 checks/calls
Street index: 0
Public cards: []
Bets: P0: 100, P1: 100.0, Chips: P0: 9900, P1: 9900.0
Player 0 goes all-in for 10000
Street index: 0
Public cards: []
Bets: P0: 10000, P1: 100.0, Chips: P0: 0, P1: 9900.0
Player 1 folds
======= New Round =======
Player 0 cards: [Qs, Td], chips: 10000.0
Player 1 cards: [Jd, 5d], chips: 9850.0
Street index: 0
Public cards: []
Bets: P0: 100, P1: 50.0, Chips: P0: 10000.0, P1: 9850.0
Player 1 bets/raises 100 to 200
Street index: 0
Public cards: []
Bets: P0: 100, P1: 200, Chips: P0: 10000.0, P1: 9700.0
Player 0 goes all-in for 9900.0
Street index: 0
Public cards: []
Bets: P0: 9900.0, P1: 200, Chips: P0: 200.0, P1: 9700.0
Player 1 does not have enough chips to raise, going all-in instead
Player 1 has lost all their chips.
Final stacks: [200